## Project Milestone 4 - Linguistic Harbingers of Betrayal Extension

First, input all the libraries we need to get the plot:

In [1]:
import json

from scipy.stats import sem
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Since the data file contains only the selected games, there is no need to consider the selection criteria when retreving the message data. 
*The condition "relationships that contain at least two consecutive and reciprocated acts of support that span at least three seasons in game time, with no more than five seasons passing between two acts of friendship" is fullfilled.

Here, we read the data from json file and get the number of games, which is 500, including 250 selected games ended up betrayal and 250 selected games ended up lasting friendship between two players.


In [2]:
# read data from diplomacy_data.json with reading mode
with open("diplomacy_data.json", "r") as f:
        data = json.load(f)
len(data)

500

Since we only consider dialogs that happen before the last act of friendship, we need to know when does the last act of friendship happen. Later when extracting feature values, extract them from the seasons that is before the last support season.

In [3]:
# a function to get the season where the last support between two players happened
# entry refer to one entry in the dataset, one game
def last_support(entry):
    seasons = entry['seasons']
    last_support = None
    for season in seasons[:-1]:
        if 'support' in season['interaction'].values():
            last_support = season['season']
    return last_support

Given the messages sent by a player from a season in a game, we need to calculate the different variable values of the messages.

Variables we consider:

1. sent_pos: Positive sentiment
2. sent_neu: Neutral sentiment
3. sent_neg: Negative sentiment
4. discourse_comp: Discourse complexity (calculated through comparison, contingency, expansive and temporal)
5. plan: Planning level (calculated through future)
6. argu_level: Argumentation level (calculated through claim and premise)
7. n_request: Number of requests
8. politeness: Politeness
9. subj: Subjectivity
10. n_words: Number of words
11. n_sentences: Number of sentences


In [4]:
# a function to get the average value across the seasons for all features of the messages
# msgs: messages sent by a player from a season in a game
def extract_features(msgs):
    n_sents = sum(m['n_sentences'] for m in msgs) * 1.0
    if(n_sents==0.0):
        print(msgs)
    
    # Sentiment
    # compute positive sentiment score
    sent_pos = sum(m['sentiment'].get("positive") for m in msgs) / len(msgs)
    # compute neutral sentiment score
    sent_neu = sum(m['sentiment'].get("neutral") for m in msgs) / len(msgs)
    # compute positive sentiment score
    sent_neg = sum(m['sentiment'].get("negative") for m in msgs) / len(msgs)
    
    # Argumentation and Discourse
    # compute discourse complexity through comparison, contingency, expansive and temporal
    comparison = sum(len(m['lexicon_words'].get("disc_comparison", []))for m in msgs) / n_sents
    contingency = sum(len(m['lexicon_words'].get("disc_contingency", []))for m in msgs) / n_sents
    expansive = sum(len(m['lexicon_words'].get("disc_expansion", []))for m in msgs) / n_sents
    temporal = sum(len(m['lexicon_words'].get("disc_temporal_rest", []))for m in msgs) / n_sents
    discourse_comp = comparison+contingency+expansive+temporal
    # compute the average number of markers refer to future
    plan = sum(len(m['lexicon_words'].get("disc_temporal_future", []))for m in msgs) / n_sents
    # compute argumentation level through claim and premise
    claim = sum(len(m['lexicon_words'].get("claim", []))for m in msgs) / n_sents
    premise = sum(len(m['lexicon_words'].get("premise", []))for m in msgs) / n_sents
    argu_level = claim+premise
    # compute average number of request
    n_request = sum(m['n_requests'] for m in msgs) / len(msgs)
    
    # Politeness
    politeness = sum(m['politeness'] for m in msgs) / len(msgs)
    
    # Subjectivity using allsubj
    subj = sum(len(m['lexicon_words'].get("allsubj", []))for m in msgs) / n_sents
    
    # Talkativeness
    # compute average number of words
    n_words = sum(m['n_words'] for m in msgs) / len(msgs)
    # compute average number of sentences
    n_sentences = sum(m['n_sentences'] for m in msgs) / len(msgs)

    return np.array([sent_pos, sent_neu, sent_neg, discourse_comp, plan, argu_level, n_request, politeness, subj, n_words, n_sentences])

From paper: To ensure that we are studying conversational patterns that occur only when the two individuals in the dyad are ostensibly being friends, we only extract features from the messages exchanged before the last act of friendship.
Therefore, the messages we extract should from seasons less than last support season.

In [5]:
# a function to get average politeness scores from dataset entries 
# data: the dataset where we extract politeness score from
# betrayal: if the games we consider end up betrayal
# betrayer: if the person to be analyzed is the (potential) betrayer
def process_data(data, betrayal, betrayer):
    results = []

    # loop in every game in data
    for entry in data:
        len_seasons = len(entry['seasons'])                   
        # a matrix to store features of different seasons of one game
        data41game = np.zeros((len_seasons, 11))
        cut_ind = 0
        # none is used to check if the season contain any valuale msg
        none = True
                        
        if(entry['betrayal'] == betrayal):
            for i, season in enumerate(entry['seasons']):
                if(season['season'] <= last_support(entry)):
                    #if(len(season['messages']['betrayer']) > 0 and len(season['messages']['victim']) > 0):                
                    if(betrayer):
                        if(len(season['messages']['betrayer']) > 0 and sum(m['n_sentences'] for m in season['messages']['betrayer'])>0):
                            none = False
                            data41game[i,:] = extract_features(season['messages']['betrayer'])
                    else:
                        if(len(season['messages']['victim']) > 0):
                            none = False
                            data41game[i,:] = extract_features(season['messages']['victim'])
                else:
                    cut_ind = i
                    break
        
        # we consider only games where friendship lasts for at least four season
        if(cut_ind>4):
            # save only the four seasons before the last support
            data41game = data41game[cut_ind-4:cut_ind,:]
            # 
            if(none==False):
                data41game = data41game.flat
                results.append(data41game)        

    # return value is a list of features of the indicated player in the indicated type of games
    return results

Get the politeness score of the four conditions below:

In [6]:
# for the games end up betrayal, get politeness score for each betrayer
betray_er = pd.DataFrame(process_data(data, True, True))

# for the games end up betrayal, get politeness score for each victim
betray_vi = pd.DataFrame(process_data(data, True, False)) 

# for the games not end up betrayal, get politeness score for each potential betrayer
control_er = pd.DataFrame(process_data(data, False, True))

# for the games not end up betrayal, get politeness score for each potential victim
control_vi = pd.DataFrame(process_data(data, False, False)) 

In [7]:
betray_er

,0,1,2,3,4,5,6,7,8,9,...,34,35,36,37,38,39,40,41,42,43
0,2.000000,2.5,2.000000,1.538462,0.307692,0.461538,5.500000,0.982703,4.384615,166.500000,...,0.666667,1.333333,0.500000,0.000000,0.166667,1.333333,0.761101,3.500000,35.666667,2.0
1,2.000000,3.0,4.500000,0.473684,0.105263,0.263158,4.500000,0.852683,2.736842,135.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.852749,1.500000,33.000000,4.0
2,0.000000,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.322627,1.000000,26.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
3,2.000000,3.5,2.500000,0.312500,0.062500,0.125000,4.500000,0.698452,0.937500,80.000000,...,2.000000,2.000000,0.000000,0.000000,0.384615,3.500000,0.554453,1.538462,50.000000,6.5
4,1.000000,3.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.911976,0.800000,32.000000,...,1.000000,3.000000,0.666667,0.333333,0.333333,2.000000,0.975159,2.166667,59.000000,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
95,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
96,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.717932,4.000000,23.000000,1.0
97,1.000000,3.5,3.500000,0.312500,0.125000,0.250000,4.500000,0.602945,2.875000,133.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0


In [8]:
betray_vi

,0,1,2,3,4,5,6,7,8,9,...,34,35,36,37,38,39,40,41,42,43
0,1.000000,3.666667,4.666667,0.178571,0.035714,0.214286,3.666667,0.607331,1.571429,120.000000,...,1.500000,3.000000,0.363636,0.0,0.181818,2.000000,0.774352,2.181818,53.500000,5.5
1,0.000000,1.500000,5.000000,2.000000,0.538462,0.538462,5.000000,0.561690,3.230769,128.000000,...,4.000000,1.000000,1.166667,0.5,0.000000,3.000000,0.939752,2.833333,89.000000,6.0
2,0.000000,0.500000,0.500000,1.500000,0.500000,0.000000,1.000000,0.603157,5.500000,22.500000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
3,0.500000,0.000000,1.000000,3.666667,1.000000,0.000000,0.000000,0.845753,3.666667,35.500000,...,1.000000,2.000000,0.000000,0.0,0.333333,2.000000,0.931822,2.333333,47.000000,3.0
4,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.456725,0.750000,10.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,0.333333,0.333333,1.666667,0.142857,0.000000,0.857143,2.000000,0.473368,2.714286,42.666667,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
91,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
92,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
93,1.000000,0.666667,1.666667,0.200000,0.000000,0.400000,2.000000,0.696383,2.600000,45.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0


In [9]:
control_er

,0,1,2,3,4,5,6,7,8,9,...,34,35,36,37,38,39,40,41,42,43
0,0.000000,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,0.000000,0.0,2.0,0.00000,0.000000,1.000000,1.000000,0.596527,3.000000,77.0,...,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.500000,0.000000,5.000000,1.0
2,0.000000,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
3,0.000000,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.241535,1.500000,17.000000,2.0
4,1.666667,3.5,2.5,0.26087,0.043478,0.065217,2.666667,0.604672,1.847826,82.0,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.362865,2.000000,4.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,1.000000,2.5,1.5,0.20000,0.000000,0.300000,2.000000,0.561422,1.700000,54.5,...,3.333333,2.666667,0.428571,0.000000,0.142857,2.666667,0.442122,1.047619,68.666667,7.0
111,0.000000,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.500000,1.000000,0.666667,0.000000,0.666667,0.500000,0.562627,2.333333,19.500000,1.5
112,0.000000,0.0,1.0,0.00000,0.000000,0.000000,1.000000,0.698595,4.000000,12.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
113,0.000000,1.0,0.0,0.00000,0.000000,0.000000,1.000000,0.770494,8.000000,36.0,...,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.208446,1.000000,14.000000,1.0


In [10]:
control_vi

,0,1,2,3,4,5,6,7,8,9,...,34,35,36,37,38,39,40,41,42,43
0,0.0,0.000000,0.0,0.00,0.000,0.000,0.0,0.000000,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
1,1.0,2.000000,1.0,0.75,0.000,0.500,2.0,0.368588,1.250000,59.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
2,0.0,0.000000,0.0,0.00,0.000,0.000,0.0,0.000000,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
3,0.0,0.000000,0.0,0.00,0.000,0.000,0.0,0.000000,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
4,0.0,0.000000,0.0,0.00,0.000,0.000,0.0,0.000000,0.000000,0.0,...,0.000000,4.0,0.000000,0.000000,0.333333,2.0,0.913732,1.833333,97.000000,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,1.0,1.000000,2.0,0.75,0.125,0.375,3.0,0.427144,4.000000,85.5,...,1.333333,2.0,0.833333,0.166667,0.500000,3.0,0.799919,2.583333,68.333333,4.0
111,0.0,0.000000,0.0,0.00,0.000,0.000,0.0,0.000000,0.000000,0.0,...,0.000000,3.0,1.000000,0.333333,0.333333,3.0,0.591054,4.000000,75.000000,3.0
112,0.0,2.666667,1.0,0.00,0.000,0.000,1.0,0.445574,1.272727,29.0,...,2.000000,4.0,0.333333,0.000000,0.000000,3.0,0.788146,1.000000,56.000000,6.0
113,0.0,2.000000,2.0,0.75,0.000,0.000,1.0,0.754371,2.750000,80.0,...,0.500000,2.5,0.625000,0.375000,0.500000,1.0,0.627844,2.250000,44.000000,4.0
